# 保存和加载模型
在本节中，我们将了解如何通过保存、加载和运行模型预测来持久化模型状态。

In [5]:
import torch
import torchvision.models as models

## 保存和加载模型权重
PyTorch 模型将学习到的参数存储在一个名为 `state_dict` 的内部状态字典中。可以通过 torch.save 方法持久化这些参数。

In [6]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

要加载模型权重，需要先创建相同模型的实例，然后使用 load_state_dict() 方法加载参数。

In [7]:
model = models.vgg16() # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

请确保在推理之前调用 model.eval() 方法，以将 dropout 和批归一化层设置为评估模式。否则将导致不一致的推理结果。

# 保存和加载具有形状的模型
加载模型权重时，我们需要先实例化模型类，因为该类定义了网络的结构。我们可能希望将此类的结构与模型一起保存，在这种情况下，我们可以将 model（而不是 model.state_dict()）传递给保存函数。

In [8]:
torch.save(model, 'model.pth')

然后我们可以像这样加载模型：

In [9]:
model = torch.load('model.pth')

# 相关教程
- [在 PyTorch 中保存和加载常规检查点](https://pytorch.ac.cn/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html)

# 保存通用检查点
不仅仅保存权重

```python
# Additional information
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)
```

# 加载通用检查点
```python
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
```

在运行推理之前，必须调用 model.eval() 将 dropout 和批量归一化层设置为评估模式。如果不这样做，将导致推理结果不一致。

如果希望继续训练，请调用 model.train() 以确保这些层处于训练模式。